In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
import os
source_path = '/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set'

source_path_no = os.path.join(source_path, 'Healthy')
source_path_yes = os.path.join(source_path, 'Brain Tumor')

# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_yes))} Tumor Images.")
print(f"There are {len(os.listdir(source_path_no))} Non Tumor Images.")

In [ ]:
import cv2
import os

# Create empty lists to store the images and their corresponding labels
images = []
labels = []

# Define the target size for resizing
target_size = (256, 256)

# Read and preprocess positive (yes) cases
for image_file in os.listdir(source_path_yes):
    image_path = os.path.join(source_path_yes, image_file)
    image = cv2.imread(image_path)
    # Resize the image to the target size (256x256)
    image = cv2.resize(image, target_size)
    # Normalize the image by cropping (center crop)
    h, w = image.shape[:2]
    crop_start_x = (w - 224) // 2
    crop_start_y = (h - 224) // 2
    image = image[crop_start_y:crop_start_y+224, crop_start_x:crop_start_x+224]
    # Convert BGR to RGB (if needed)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Append the processed image and its label to the lists
    images.append(image)
    labels.append(1)  # 1 for positive cases

# Read and preprocess negative (no) cases
for image_file in os.listdir(source_path_no):
    image_path = os.path.join(source_path_no, image_file)
    image = cv2.imread(image_path)
    # Resize the image to the target size (256x256)
    image = cv2.resize(image, target_size)
    # Normalize the image by cropping (center crop)
    h, w = image.shape[:2]
    crop_start_x = (w - 224) // 2
    crop_start_y = (h - 224) // 2
    image = image[crop_start_y:crop_start_y+224, crop_start_x:crop_start_x+224]
    # Convert BGR to RGB (if needed)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Append the processed image and its label to the lists
    images.append(image)
    labels.append(0)  # 0 for negative cases

# Convert the image and label lists to NumPy arrays for machine learning
X = np.array(images)
y = np.array(labels)

In [ ]:
# Display examples of "Brain Tumor" images
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle("Examples of 'Positive and Negative' Image Classes", fontsize=16)
for i in range(3):
    axes[0, i].imshow(X[y == 1][i])  # Display the first 3 "yes" images
    axes[0, i].set_title("Positive Case")
    axes[0, i].axis('off')

# Display examples of "Healthy" images
for i in range(3):
    axes[1, i].imshow(X[y == 0][i])  # Display the first 3 "no" images
    axes[1, i].set_title("Negative Case")
    axes[1, i].axis('off')

plt.show()

In [ ]:
source_tumor_dir = '/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor'
source_healthy_dir = '/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Healthy'



In [ ]:
# Create the train and test directories


train_directory = '/kaggle/working/train'
test_directory = '/kaggle/working/test'
validation_directory = '/kaggle/working/validation'

os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)
os.makedirs(validation_directory, exist_ok=True)

# Define the destination directories for train, validation, and test sets
train_healthy_dir = '/kaggle/working/train/healthy'
validation_healthy_dir = '/kaggle/working/validation/healthy'
test_healthy_dir = '/kaggle/working/test/healthy'

train_tumor_dir = '/kaggle/working/train/tumor'
validation_tumor_dir = '/kaggle/working/validation/tumor'
test_tumor_dir = '/kaggle/working/test/tumor'

# Create destination directories if they don't exist
for directory in [train_healthy_dir, validation_healthy_dir, test_healthy_dir,
                  train_tumor_dir, validation_tumor_dir, test_tumor_dir]:
    os.makedirs(directory, exist_ok=True)

def copy_and_split_images(source_dir, train_dir, validation_dir, test_dir, split_ratio=(0.8, 0.1)):
    # List of image files in the source directory
    source_files = os.listdir(source_dir)
    # Shuffle the list to randomize
    random.shuffle(source_files)

    # Calculate split points
    train_split = int(len(source_files) * split_ratio[0])
    validation_split = train_split + int(len(source_files) * split_ratio[1])

    # Split the files into train, validation, and test sets
    train_files = source_files[:train_split]
    validation_files = source_files[train_split:validation_split]
    test_files = source_files[validation_split:]

    # Copy train files to the train directory
    for file in train_files:
        src = os.path.join(source_dir, file)
        dst = os.path.join(train_dir, file)
        shutil.copy(src, dst)

    # Copy validation files to the validation directory
    for file in validation_files:
        src = os.path.join(source_dir, file)
        dst = os.path.join(validation_dir, file)
        shutil.copy(src, dst)

    # Copy test files to the test directory
    for file in test_files:
        src = os.path.join(source_dir, file)
        dst = os.path.join(test_dir, file)
        shutil.copy(src, dst)



In [ ]:
import random
import os
import shutil
# Copy and split tumor images
copy_and_split_images(source_tumor_dir, train_tumor_dir, validation_tumor_dir, test_tumor_dir)

# Copy and split healthy images
copy_and_split_images(source_healthy_dir, train_healthy_dir, validation_healthy_dir, test_healthy_dir)

In [ ]:
print(f"There are {len(os.listdir(train_tumor_dir))} images of tumor for training")
print(f"There are {len(os.listdir(train_healthy_dir))} non tumor for training")
print(f"There are {len(os.listdir(validation_tumor_dir))} images of tumor for validation")
print(f"There are {len(os.listdir(validation_healthy_dir))} non tumor for validation")
print(f"There are {len(os.listdir(test_healthy_dir))} non tumor for testing")
print(f"There are {len(os.listdir(test_tumor_dir))} tumor images for testing")

In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip= True,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  zoom_range = 0.2,
                                  )

test_datagen = ImageDataGenerator()




In [ ]:
train_generator = train_datagen.flow_from_directory(train_directory, 
                                       target_size= (224,224), 
                                       class_mode= 'binary',
                                       color_mode= "rgb", 
                                       shuffle= True, 
                                       batch_size= 32)

valid_generator = test_datagen.flow_from_directory(validation_directory, 
                                       target_size= (224,224), 
                                       class_mode= 'binary',
                                       color_mode= "rgb", 
                                       shuffle= True, 
                                       batch_size= 32)

test_generator = test_datagen.flow_from_directory(test_directory, 
                                       target_size= (224,224), 
                                       class_mode= 'binary',
                                       color_mode= "rgb", 
                                       shuffle= False, 
                                       batch_size= 32)


In [ ]:
base_model = tf.keras.applications.VGG16(
    include_top=False,  # Exclude the fully connected layers
    weights='imagenet',
    input_shape=(224, 224, 3)
)

In [ ]:
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D ,BatchNormalization

vgg_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

vgg_model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10 ,restore_best_weights = True)
checkpoint = ModelCheckpoint("Brain_Tumor_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
vgg_history = vgg_model.fit(train_generator,verbose=1,callbacks = [early_stopping,checkpoint],epochs=50,validation_data=valid_generator)

In [ ]:
 y_true_labels = test_generator.classes

y_pred = vgg_model.predict(test_generator)

threshold = 0.5  

# Convert the raw predictions to binary predictions
y_pred_binary = (y_pred > threshold).astype(int)

# Print the binary predictions
print(classification_report(y_pred_binary, y_true_labels))

In [ ]:
confusion_matrix(y_pred_binary, y_true_labels)

In [ ]:
from PIL import Image
import numpy as np
from ipywidgets import FileUpload, Output, VBox
import io
import tensorflow as tf
from tensorflow.keras.models import load_model

# Create a file upload widget
upload = FileUpload()

# Create an output widget to display predictions
output = Output()

# Load your model
my_model = load_model('Brain_Tumor_Model.h5')

# Set a threshold for binary classification
threshold = 0.5

# Function to handle file upload and prediction
def handle_upload(change):
    with output:
        # Clear previous output
        output.clear_output()

        # Get the uploaded file content
        uploaded_file_content = list(upload.value.values())[0]["content"]

        # Process the file content as needed (e.g., convert to image array)
        img = Image.open(io.BytesIO(uploaded_file_content))

        # Normalize the image by cropping (center crop)
        h, w = img.size
        crop_start_x = (w - 224) // 2
        crop_start_y = (h - 224) // 2
        img = img.crop((crop_start_x, crop_start_y, crop_start_x + 224, crop_start_y + 224))
        img = img.resize((224, 224))

        # Convert the image to a format suitable for model prediction
        #img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
        img_array = np.expand_dims(img, axis=0)

        # Make predictions using your model
        predictions = my_model.predict(img_array)

        # Convert predictions to binary (0 or 1) based on the threshold
        binary_prediction = 'Tumor Detected' if predictions[0][0] > threshold else 'No Tumor Detected'

        # Print or use the binary prediction as needed
        print("Prediction:", binary_prediction)
        
        

# Attach the handle_upload function to the change event of the upload widget
upload.observe(handle_upload, names='_counter')

# Display the widgets
VBox([upload, output])

In [ ]:
from IPython.display import FileLink, FileLinks
from google.colab import files

# Add an upload button
uploaded = files.upload()

# Print the names of the uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


In [ ]:
def predict_input_image(img):
    # Normalize the image by cropping (center crop)
    h, w = img.shape[:2]
    crop_start_x = (w - 224) // 2
    crop_start_y = (h - 224) // 2
    img = img[crop_start_y:crop_start_y+224, crop_start_x:crop_start_x+224]
    img = tf.image.resize(img, [224,224])
    img = np.expand_dims(img, axis = 0)
    
    # Make predictions
    model = tf.keras.models.load_model('Tumor_Model.h5')
    prediction = model.predict(img)
    result = 'No Tumor Detected' if prediction[0][0] > 0.5 else 'Tumor detected'
    

    return prediction